In [44]:
import os

os.environ['LANGCHAIN_TRACING_V2']="true"
os.environ['LANGCHAIN_API_KEY']='ls__971f18068e7e4536a62f1811c3aeb059'

In [45]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

In [46]:
llm.invoke("how can langsmith help with testing?")

'Langsmith is a language model developed by Google that can assist with testing in several ways:\n\n1. Code Generation: Langsmith can generate code snippets based on natural language inputs, which can be used for testing purposes. For example, you can give Langsmith a description of a functionality you want to test, and it will generate the corresponding test cases.\n2. Test Case Generation: Langsmith can also generate test cases automatically based on the API documentation of a system or application. This can save developers a lot of time and effort in creating test cases manually.\n3. Code Review: Langsmith can review code changes and suggest improvements, which can help identify potential issues before they become bugs. This can be especially useful during testing, as it can help catch errors early on.\n4. Bug Reporting: If a bug is found during testing, Langsmith can assist in creating a clear and concise report that includes the necessary details to reproduce the issue. This can s

In [47]:
# Prompt template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

In [48]:
# Chaining
chain = prompt | llm 

In [49]:
chain.invoke({"input": "how can langsmith help with testing?"})

"\nAs a world-class technical documentation writer, I must say that LinguaSmith is an incredible tool for testing and proofreading your content. Here are some ways in which Langsmith can help with testing:\n\n1. Grammar and Spelling Checks: Langsmith's advanced grammar and spelling checks can identify even the most subtle errors in your content, ensuring that it is free of mistakes and typos.\n2. Contextual Error Detection: Langsmith's contextual error detection capabilities allow it to identify errors that may not be caught by traditional grammar and spell checkers. For example, it can detect errors in sentence structure, punctuation, and word choice.\n3. Consistency Checking: Langsmith can help you maintain consistency throughout your content by checking for consistent use of terminology, formatting, and style.\n4. Content Reuse Detection: If you have a large volume of content to test, Langsmith can help identify duplicate or redundant content, allowing you to simplify or consolidate

In [50]:
# Output parsing
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [51]:
chain = prompt | llm | output_parser

### Retrieval Chain

In [52]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com")

docs = loader.load()

In [53]:
# Using vector DB
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

In [54]:
# Semantic search using FAISS
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

Now that we have this data indexed in a vectorstore, we will create a retrieval chain. This chain will take an incoming question, look up relevant documents, then pass those documents along with the original question into an LLM and ask it to answer the original question.

First, let's set up the chain that takes a question and the retrieved documents and generates an answer.

In [55]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

If we wanted to, we could run this ourselves by passing in documents directly:

In [56]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'\nBased on the provided context, Langsmith can help with testing by allowing users to visualize their test results.'

However, we want the documents to first come from the retriever we just set up. That way, for a given question we can use the retriever to dynamically select the most relevant documents and pass those in.

In [57]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [58]:
# We can now invoke this chain. This returns a dictionary - the response from the LLM is in the answer key

response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

Based on the provided context, LangSmith can help with testing by providing a platform for building and debugging production-grade LLM applications. Specifically, it offers the following testing capabilities:

1. Evaluation: LangSmith provides an evaluation capability that allows you to test your LLM applications in a controlled environment before deploying them in production.
2. Monitoring: LangSmith offers monitoring capabilities that enable you to track the performance and behavior of your LLM applications in real-time, allowing you to identify and fix issues quickly.
3. Tracing: LangSmith provides tracing capabilities that allow you to debug and test your LLM applications by visualizing their execution path and identifying potential errors or bottlenecks.
4. Prompt Hub: LangSmith offers a prompt management tool called the Prompt Hub, which allows you to manage and organize your prompts for your LLM applications, making it easier to test and evaluate them.

Overall, LangSmith provid

### Conversation Retrieval Chain

The chain we've created so far can only answer single questions. One of the main types of LLM applications that people are building are chat bots. So how do we turn this chain into one that can answer follow up questions?

We can still use the create_retrieval_chain function, but we need to change two things:

The retrieval method should now not just work on the most recent input, but rather should take the whole history into account.
The final LLM chain should likewise take the whole history into account


Updating Retrieval

In order to update retrieval, we will create a new chain. This chain will take in the most recent input (input) and the conversation history (chat_history) and use an LLM to generate a search query.

In [59]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

We can test this out by passing in an instance where the user is asking a follow up question.

In [60]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content="LangSmith | ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith\n\n\n\n\n\nSkip to main contentğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubLangSmithOn this pageLangSmithIntroductionâ€‹LangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Startâ€‹Tracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Stepsâ€‹Check out the following sections to learn more about LangSmith:User Guide: Learn about the workflows LangSmith supports at each stage of the LLM application lifecy

You should see that this returns documents about testing in LangSmith. This is because the LLM generated a new query, combining the chat history with the follow up question.

Now that we have this new retriever, we can create a new chain to continue the conversation with these retrieved documents in mind.

In [61]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

We can now test this out end-to-end:

In [62]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content="LangSmith | ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith\n\n\n\n\n\nSkip to main contentğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubLangSmithOn this pageLangSmithIntroductionâ€‹LangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Startâ€‹Tracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Stepsâ€‹Check ou

### Agent

We've so far create examples of chains - where each step is known ahead of time. The final thing we will create is an agent - where the LLM decides what steps to take.

NOTE: for this example we will only show how to create an agent using OpenAI models, as local models are not reliable enough yet.

One of the first things to do when building an agent is to decide what tools it should have access to. For this example, we will give the agent access to two tools:

1. The retriever we just created. This will let it easily answer questions about LangSmith
2. A search tool. This will let it easily answer questions that require up to date information.

First, let's set up a tool for the retriever we just created:

In [63]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

The search tool that we will use is Tavily. This will require an API key (they have generous free tier). After creating it on their platform, you need to set it as an environment variable:

In [64]:
from langchain_community.tools.tavily_search import TavilySearchResults

os.environ['TAVILY_API_KEY'] = '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'
search = TavilySearchResults()

In [65]:
tools = [retriever_tool, search]

Now that we have the tools, we can create an agent to use them. We will go over this pretty quickly - for a deeper dive into what exactly is going on, check out the Agent's Getting Started documentation

Now we can use it to get a predefined prompt

In [66]:
# from langchain_openai import ChatOpenAI
# from langchain import hub
# from langchain.agents import create_openai_functions_agent, cr
# from langchain.agents import AgentExecutor

# # Get the prompt to use - you can modify this!
# # prompt = hub.pull("hwchase17/openai-functions-agent")
# prompt = hub.pull("homanp/superagent")
# # llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
# agent = create_openai_functions_agent(llm, tools, prompt)
# agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

We can now invoke the agent and see how it responds! We can ask it questions about LangSmith:

In [67]:
# agent_executor.invoke({"input": "how can langsmith help with testing?"})

### LangServe

#### Server
To create a server for our application we'll make a serve.py file. This will contain our logic for serving our application. It consists of three things:

The definition of our chain that we just built above
Our FastAPI app
A definition of a route from which to serve the chain, which is done with langserve.add_routes